## Importing Required Libraries

In [2]:
import keras
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


# Defining the path to Model and Images

In [9]:
IMAGE_SIZE = (256,256,3)

path_base_model = './/models//'
path_base_input = './/Pneumonia//'
path_base_mask = './/Pneumonia_Mask//'

# Model loads

In [3]:
BACKBONE = 'efficientnetb0'
model1 = sm.Unet(BACKBONE, input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],IMAGE_SIZE[2]),classes=1, activation='sigmoid',encoder_weights='imagenet')
model2 = sm.Unet(BACKBONE, input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],IMAGE_SIZE[2]),classes=1, activation='sigmoid',encoder_weights='imagenet')
model3 = sm.Unet(BACKBONE, input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],IMAGE_SIZE[2]),classes=1, activation='sigmoid',encoder_weights='imagenet')

BACKBONE = 'efficientnetb7'
model4 = sm.Unet(BACKBONE, input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],IMAGE_SIZE[2]),classes=1, activation='sigmoid',encoder_weights='imagenet')
model5 = sm.Unet(BACKBONE, input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],IMAGE_SIZE[2]),classes=1, activation='sigmoid',encoder_weights='imagenet')

preprocess_input = sm.get_preprocessing(BACKBONE)

model1.load_weights(path_base_model + 'model1.hdf5')
model2.load_weights(path_base_model + 'model2.hdf5')
model3.load_weights(path_base_model + 'model3.hdf5')
model4.load_weights(path_base_model + 'model4.hdf5')
model5.load_weights(path_base_model + 'model5.hdf5')

# Functions

In [5]:
def preprocessing_HE(img_):
    
    hist, bins = np.histogram(img_.flatten(), 256,[0,256])
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')

    img_2 = cdf[img_]
    
    return img_2  
        
def get_binary_mask (mask_, th_ = 0.5):
    mask_[mask_>th_]  = 1
    mask_[mask_<=th_] = 0
    return mask_
    
def ensemble_results (mask1_, mask2_, mask3_, mask4_, mask5_):
    
    mask1_ = get_binary_mask (mask1_)
    mask2_ = get_binary_mask (mask2_)
    mask3_ = get_binary_mask (mask3_)
    mask4_ = get_binary_mask (mask4_)
    mask5_ = get_binary_mask (mask5_)
    
    ensemble_mask = mask1_ + mask2_ + mask3_ + mask4_ + mask5_
    ensemble_mask[ensemble_mask<=2.0] = 0
    ensemble_mask[ensemble_mask> 2.0] = 1
    
    return ensemble_mask

def postprocessing_HoleFilling (mask_):
    
    ensemble_mask_post_temp = ndimage.binary_fill_holes(mask_).astype(int)
     
    return ensemble_mask_post_temp

def get_maximum_index (labeled_array):
    
    ind_nums = []
    for i in range (len(np.unique(labeled_array)) - 1):
        ind_nums.append ([0, i+1])
        
    for i in range (1, len(np.unique(labeled_array))):
        ind_nums[i-1][0] = len(np.where (labeled_array == np.unique(labeled_array)[i])[0])
        
    ind_nums = sorted(ind_nums)
    
    return ind_nums[len(ind_nums)-1][1], ind_nums[len(ind_nums)-2][1]
    
def postprocessing_EliminatingIsolation (ensemble_mask_post_temp):
        
    labeled_array, num_features = label(ensemble_mask_post_temp)
    
    ind_max1, ind_max2 = get_maximum_index (labeled_array)
    
    ensemble_mask_post_temp2 = np.zeros (ensemble_mask_post_temp.shape)
    ensemble_mask_post_temp2[labeled_array == ind_max1] = 1
    ensemble_mask_post_temp2[labeled_array == ind_max2] = 1    
    
    return ensemble_mask_post_temp2.astype(int)

def get_prediction(model_, img_org_):
    
    img_org_resize = cv2.resize(img_org_,(IMAGE_SIZE[0],IMAGE_SIZE[1]),cv2.INTER_AREA)
    img_org_resize_HE = preprocessing_HE (img_org_resize)    
    img_ready = preprocess_input (img_org_resize_HE)

    img_ready = np.expand_dims(img_ready, axis=0) 
    pr_mask = model_.predict(img_ready)
    pr_mask = np.squeeze(pr_mask)
    pr_mask = np.expand_dims(pr_mask, axis=-1)    
    return pr_mask[:,:,0]

# Inference

In [10]:
for path_ in sorted(glob.glob (path_base_input + '*.*')):

    print ('file: ', path_.split('/')[-1])
    
    img = cv2.imread(path_)    
    pr_mask1 = get_prediction (model1, img)
    pr_mask2 = get_prediction (model2, img)
    pr_mask3 = get_prediction (model3, img)
    pr_mask4 = get_prediction (model4, img)
    pr_mask5 = get_prediction (model5, img)   
    
    ensemble_mask            = ensemble_results (pr_mask1, pr_mask2, pr_mask3, pr_mask4, pr_mask5)
    ensemble_mask_post_HF    = postprocessing_HoleFilling (ensemble_mask)
    ensemble_mask_post_HF_EI = postprocessing_EliminatingIsolation (ensemble_mask_post_HF)
    
    image_name = path_.split('\\')[-1]
    cv2.imwrite(path_base_mask+image_name,ensemble_mask_post_HF_EI*255)
    
#     plt.figure(figsize=(20, 18))
#     plt.subplot(3, 3, 1)
#     plt.title('Input with HE')
#     plt.imshow(img)
#     plt.subplot(3, 3, 2)
#     plt.title('Model 1')
#     plt.imshow(pr_mask1, cmap='gray')
#     plt.subplot(3, 3, 3)
#     plt.title('Model 2')
#     plt.imshow(pr_mask2, cmap='gray')
#     plt.subplot(3, 3, 4)
#     plt.title('Model 3')
#     plt.imshow(pr_mask3, cmap='gray')
#     plt.subplot(3, 3, 5)
#     plt.title('Model 4')
#     plt.imshow(pr_mask4, cmap='gray')
#     plt.subplot(3, 3, 6)
#     plt.title('Model 5')
#     plt.imshow(pr_mask5, cmap='gray')
#     plt.subplot(3, 3, 7)
#     plt.title('Ensemble')
#     plt.imshow(ensemble_mask, cmap='gray')
#     plt.subplot(3, 3, 8)
#     plt.title('Ensemble + HF')
#     plt.imshow(ensemble_mask_post_HF, cmap='gray')
#     plt.subplot(3, 3, 9)
#     plt.title('Ensemble + HF + EI')
#     plt.imshow(ensemble_mask_post_HF_EI, cmap='gray')
#     plt.show()

file:  Pneumonia\person1000_bacteria_2931.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1000_virus_1681.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1001_bacteria_2932.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1002_bacteria_2933.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1003_bacteria_2934.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1003_virus_1685.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1004_bacteria_2935.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1004_virus_1686.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1005_bacteria_2936.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1005_virus_1688.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1006_bacteria_2937.

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1026_virus_1718.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1028_bacteria_2959.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1028_bacteria_2960.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1029_bacteria_2961.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1029_virus_1721.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person102_bacteria_487.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person102_virus_189.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1030_virus_1722.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1031_bacteria_2963.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1031_bacteria_2964.jpeg
1/1 [==============================] - 3s 3s/s

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1040_virus_1735.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1041_bacteria_2975.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1041_virus_1736.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1042_virus_1737.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1043_bacteria_2977.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1043_virus_1738.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1044_bacteria_2978.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1044_virus_1740.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1045_bacteria_2979.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1045_virus_1741.jpeg
1/1 [==============================] - 3s 3s/ste

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1056_bacteria_2990.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1056_virus_1755.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1057_bacteria_2991.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1057_virus_1756.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1058_bacteria_2992.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1058_virus_1757.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1059_bacteria_2993.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1059_virus_1758.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person105_virus_192.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person105_virus_193.jpeg
1/1 [==============================] - 3s 3s/step
fi

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1071_bacteria_3005.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1071_virus_1774.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1072_bacteria_3006.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1072_bacteria_3007.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1072_virus_1775.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1073_bacteria_3008.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1073_bacteria_3011.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1073_virus_1776.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1074_bacteria_3012.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person1074_bacteria_3014.jpeg
1/1 [==============================] - 

...

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person88_virus_165.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person88_virus_166.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person88_virus_167.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person890_bacteria_2814.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person890_virus_1540.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person891_virus_1541.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person892_bacteria_2817.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person893_bacteria_2818.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person894_bacteria_2819.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person894_virus_1546.jpeg
1/1 [==============================] - 2847s 2847s/step
file:  P

1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person904_bacteria_2829.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person905_bacteria_2830.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person905_virus_1561.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person906_bacteria_2831.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person906_virus_1562.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person907_bacteria_2832.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person907_virus_1563.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person908_virus_1564.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person909_bacteria_2834.jpeg
1/1 [==============================] - 2s 2s/step
file:  Pneumonia\person909_virus_1565.jpeg
1/1 [==============================] - 2s 2s/step
file:

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person931_bacteria_2856.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person931_virus_1592.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person932_virus_1593.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person933_bacteria_2858.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person933_virus_1594.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person934_bacteria_2859.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person934_virus_1595.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person935_virus_1597.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person936_bacteria_2861.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person936_virus_1598.jpeg
1/1 [==============================] - 3s 3s/step
file:  P

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person958_bacteria_2883.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person958_virus_1630.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person959_bacteria_2884.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person95_bacteria_463.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person95_virus_177.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person960_bacteria_2885.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person960_virus_1633.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person961_bacteria_2886.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person961_virus_1634.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person962_bacteria_2887.jpeg
1/1 [==============================] - 3s 3s/step
file: 

1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person982_bacteria_2909.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person982_virus_1658.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person983_bacteria_2910.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person983_virus_1660.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person984_bacteria_2911.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person985_bacteria_2912.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person986_bacteria_2913.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person987_bacteria_2914.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person988_bacteria_2915.jpeg
1/1 [==============================] - 3s 3s/step
file:  Pneumonia\person988_virus_1666.jpeg
1/1 [==============================] - 3s 3s/step